## 05 RESNET

In [29]:
import torch
from PIL import Image, ImageOps
import pickle
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import Dataset, DataLoader
import time
import copy

In [30]:
name_right = 'nochers_5x130_right' #enter name_right of the pickle file
with open(name_right, 'rb') as f:
    image_paths_right = pickle.load(f)

name_left = 'nochers_5x130_left' #enter name_left of the pickle file
with open(name_left, 'rb') as f:
    image_paths_left = pickle.load(f)

In [31]:
model = models.resnet18(pretrained = True)

#model preformulation - we have 5 classes
model.fc = nn.Linear(512,5)

#CPU to GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

#loss
criterion = nn.CrossEntropyLoss()

#optimizer
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

#decay lr by 0.1 for every 7 epochs
schedular = lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

In [32]:
preprocess= transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [33]:
def Transformator(paths, left = False):
    categories = 5
    samples_per_category = len(paths[0])
    data = torch.zeros((samples_per_category*categories, 3, 224, 224))
    labels = []

    for i in range(samples_per_category):
        for j in range(categories):
            path = paths[j][i]
            input_image = Image.open(path)
            if left:
                input_image = ImageOps.mirror(input_image)
            inx = i*categories+j
            if inx % 50 == 0 and j == 0:
                print(f'{inx}/{categories*samples_per_category}')
            try:
                input_tensor = preprocess(input_image)
                data[inx] = input_tensor
            except:
                print(inx, path)
                print('Mistake')
            labels.append(j)
    return data, labels

In [34]:
rights, labels_r = Transformator(image_paths_right)
lefts, labels_l = Transformator(image_paths_left, left = True)
data=torch.cat((rights,lefts),0)
labels = labels_r + labels_l

0/650
50/650
100/650
150/650
200/650
250/650
300/650
350/650
400/650
450/650
500/650
550/650
600/650
0/650
50/650
100/650
150/650
200/650
250/650
300/650
350/650
400/650
450/650
500/650
550/650
600/650


In [39]:
labels = np.array(labels)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2)

In [41]:
class Data(Dataset):
    
    def __init__(self, data, labels):
        self.x = data
        self.y = torch.from_numpy(labels)
        self.samples = data.shape[0]
        self.features = data.shape[1]
        
    def __getitem__(self, index):
        # define indexing
        return self.x[index], self.y[index]
        
    def __len__(self):
        # len(dataset)
        return self.samples

In [42]:
train = Data(X_train, y_train)
test = Data(X_test, y_test)
print(f'train: {len(train)}, test: {len(test)}')
train.y.size()

train: 1040, test: 260


torch.Size([1040])

In [43]:
datasets = {'train': train, 'test': test}
dataloaders = {x: DataLoader(datasets[x], batch_size=5, num_workers=0) for x in ['train', 'test']}
dataset_sizes = {x: len(datasets[x]) for x in ['train', 'test']}
class_names = np.unique(labels)

In [44]:
print(dataset_sizes)
class_names

{'train': 1040, 'test': 260}


array([0, 1, 2, 3, 4])

In [45]:
def train_model(model, criterion, optimizer, scheduler, num_epochs = 10):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'test']:
            print(f'phase: {phase}')
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0

            batch_number=0
            for inputs, labels in dataloaders[phase]:
                #if batch_number%5==0:
                    #print(f'batch munber: {batch_number}')
                batch_number+=1
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels.long())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model


In [46]:
train_model(model, criterion, optimizer, schedular, num_epochs = 10)

Epoch 1/10
----------
phase: train
train Loss: 1.7245 Acc: 0.2952
phase: test
test Loss: 1.5974 Acc: 0.3192

Epoch 2/10
----------
phase: train
train Loss: 1.3770 Acc: 0.4346
phase: test
test Loss: 1.5903 Acc: 0.4000

Epoch 3/10
----------
phase: train
train Loss: 1.0642 Acc: 0.5625
phase: test
test Loss: 1.7278 Acc: 0.4077

Epoch 4/10
----------
phase: train
train Loss: 0.8060 Acc: 0.7019
phase: test
test Loss: 2.2139 Acc: 0.3885

Epoch 5/10
----------
phase: train
train Loss: 0.5458 Acc: 0.7952
phase: test
test Loss: 2.5162 Acc: 0.3808

Epoch 6/10
----------
phase: train
train Loss: 0.5937 Acc: 0.7788
phase: test
test Loss: 2.6709 Acc: 0.4115

Epoch 7/10
----------
phase: train
train Loss: 0.5822 Acc: 0.7740
phase: test
test Loss: 3.2944 Acc: 0.4115

Epoch 8/10
----------
phase: train
train Loss: 0.3011 Acc: 0.8990
phase: test
test Loss: 1.7157 Acc: 0.4923

Epoch 9/10
----------
phase: train
train Loss: 0.0831 Acc: 0.9904
phase: test
test Loss: 1.7264 Acc: 0.4923

Epoch 10/10
-------

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  